In [ ]:
import re
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from datetime import datetime
import requests

In [ ]:
# Path to auth log (works on Unix/Linux)
LOG_PATH = "/var/log/auth.log"

# Store parsed results
failed_logins = []


In [ ]:
# Regex to parse log lines
pattern = re.compile(r'^(\w{3}\s+\d+\s[\d:]+)\s[\w-]+\s.*sshd.*Failed password.*from\s([\d.]+)')

with open(LOG_PATH, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        match = pattern.search(line)
        if match:
            timestamp_str, ip = match.groups()
            timestamp = datetime.strptime(timestamp_str, "%b %d %H:%M:%S")
            failed_logins.append({'time': timestamp, 'ip': ip})


In [ ]:
df = pd.DataFrame(failed_logins)
df.head()

In [ ]:
# Count failed login attempts per IP
ip_counts = df['ip'].value_counts()
print("Top 10 suspicious IPs:\n")
print(ip_counts.head(10))

In [ ]:
# Visualize attempts over time
df.set_index('time').resample('H').count()['ip'].plot(figsize=(10,4), title='Failed Logins Over Time', grid=True)
plt.ylabel("Attempts")
plt.show()

In [ ]:
# Detect brute-force attacks (e.g., > 5 failed logins)
suspicious_ips = ip_counts[ip_counts > 5]
print("\nIPs with >5 failed attempts:\n", suspicious_ips)